<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [4]:
## install finrl library
!pip3 install git+https://github.com/AI4Finance-Foundation/FinRL-Library.git

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Cloning https://github.com/AI4Finance-Foundation/FinRL-Library.git to /private/var/folders/9y/9ztxqzcx64j6np_fxsty2gy80000gn/T/pip-req-build-r_8p1n74
  Running command git clone --filter=blob:none -q https://github.com/AI4Finance-Foundation/FinRL-Library.git /private/var/folders/9y/9ztxqzcx64j6np_fxsty2gy80000gn/T/pip-req-build-r_8p1n74
Username for 'https://github.com': ^C
ERROR: Operation cancelled by user


In [5]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [27]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

import warnings
warnings.filterwarnings('ignore')

<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [24]:
# from config.py TRAIN_START_DATE is a string
config.TRAIN_START_DATE

'2014-01-01'

In [25]:
# from config.py TRAIN_END_DATE is a string
config.TRAIN_END_DATE

'2020-07-31'

In [28]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [29]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [31]:
df.shape

(94360, 8)

In [32]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2008-12-31,3.070357,3.133571,3.047857,2.606277,607541200,AAPL,2
1,2008-12-31,57.110001,58.220001,57.060001,44.277359,6287200,AMGN,2
2,2008-12-31,17.969999,18.750000,17.910000,14.908462,9625600,AXP,2
3,2008-12-31,41.590000,43.049999,41.500000,32.005894,5443100,BA,2
4,2008-12-31,43.700001,45.099998,43.700001,30.628826,6277400,CAT,2


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [33]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index


In [34]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [35]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2008-12-31,AAPL,3.070357,3.133571,3.047857,2.606277,607541200.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,2.606277,2.606277,40.0,0.0
1,2008-12-31,AMGN,57.110001,58.220001,57.060001,44.277359,6287200.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,44.277359,44.277359,40.0,0.0
2,2008-12-31,AXP,17.969999,18.750000,17.910000,14.908462,9625600.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,14.908462,14.908462,40.0,0.0
3,2008-12-31,BA,41.590000,43.049999,41.500000,32.005894,5443100.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,32.005894,32.005894,40.0,0.0
4,2008-12-31,CAT,43.700001,45.099998,43.700001,30.628826,6277400.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,30.628826,30.628826,40.0,0.0
5,2008-12-31,CRM,7.712500,8.130000,7.707500,8.002500,5367600.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,8.002500,8.002500,40.0,0.0
6,2008-12-31,CSCO,16.180000,16.549999,16.120001,11.787783,37513700.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,11.787783,11.787783,40.0,0.0
7,2008-12-31,CVX,72.900002,74.629997,72.900002,43.670765,9964300.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,43.670765,43.670765,40.0,0.0
8,2008-12-31,DIS,22.570000,22.950001,22.520000,19.538343,9012100.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,19.538343,19.538343,40.0,0.0
9,2008-12-31,GS,82.239998,86.150002,81.120003,69.224159,14894100.0,2.0,0.0,2.921925,2.455526,100.0,66.666667,100.0,69.224159,69.224159,40.0,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [36]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

83897
9744


In [37]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2892,2020-06-30,UNH,288.570007,296.450012,287.660004,287.776794,2932900.0,1.0,-0.019479,303.925881,271.251234,52.413047,-25.838416,1.846804,288.020685,281.001438,30.43,12.918702
2892,2020-06-30,V,191.490005,193.750000,190.160004,191.101532,9040100.0,1.0,1.050791,199.130115,185.394821,53.021034,-51.532443,2.013358,191.850757,182.024673,30.43,12.918702
2892,2020-06-30,VZ,54.919998,55.290001,54.360001,50.376743,17414800.0,1.0,-0.437110,53.918423,48.729326,48.097043,-51.018266,8.508886,51.012122,51.464679,30.43,12.918702
2892,2020-06-30,WBA,42.119999,42.580002,41.759998,39.494282,4782100.0,1.0,-0.084973,43.109827,36.915700,48.830191,-14.474237,1.500723,39.594901,39.392492,30.43,12.918702
2892,2020-06-30,WMT,119.220001,120.129997,118.540001,116.121780,6836400.0,1.0,-0.886570,119.473764,113.510450,48.159686,-69.938632,3.847271,117.787628,119.723276,30.43,12.918702


In [38]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,89.904594,110737200.0,2.0,3.014603,92.699319,80.179953,62.807144,107.491815,29.730532,83.933767,77.717542,28.620001,53.068508
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,242.083481,6575800.0,2.0,3.665611,234.264608,201.821546,61.279648,271.571268,46.806139,216.584930,217.666556,28.620001,53.068508
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.086372,3301000.0,2.0,-0.389163,110.423942,87.759330,48.504813,-66.328730,3.142448,97.244635,90.695524,28.620001,53.068508
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.068508
0,2020-07-01,CAT,129.380005,129.399994,125.879997,120.651634,2807800.0,2.0,1.272629,130.624290,113.353123,52.865416,35.597277,14.457404,119.412839,113.646677,28.620001,53.068508


In [39]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [40]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 29, State Space: 291


In [41]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [42]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [43]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [44]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [45]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

---------------------------------------
| time/                 |             |
|    fps                | 76          |
|    iterations         | 100         |
|    time_elapsed       | 6           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -41.2       |
|    explained_variance | -1.19e-07   |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | 57.8        |
|    reward             | -0.45638013 |
|    std                | 1           |
|    value_loss         | 3.26        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 78         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -41.2      |
|    explained_variance | 0          |
|    lear

------------------------------------
| time/                 |          |
|    fps                | 84       |
|    iterations         | 1500     |
|    time_elapsed       | 88       |
|    total_timesteps    | 7500     |
| train/                |          |
|    entropy_loss       | -41.5    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 1499     |
|    policy_loss        | 110      |
|    reward             | 7.594344 |
|    std                | 1.01     |
|    value_loss         | 17.7     |
------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 84         |
|    iterations         | 1600       |
|    time_elapsed       | 94         |
|    total_timesteps    | 8000       |
| train/                |            |
|    entropy_loss       | -41.6      |
|    explained_variance | 1.19e-07   |
|    learning_rate      | 0.0007     |
|    n_updates    

-------------------------------------
| time/                 |           |
|    fps                | 83        |
|    iterations         | 2900      |
|    time_elapsed       | 174       |
|    total_timesteps    | 14500     |
| train/                |           |
|    entropy_loss       | -41.7     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 2899      |
|    policy_loss        | -109      |
|    reward             | 3.3146346 |
|    std                | 1.02      |
|    value_loss         | 8.39      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 83         |
|    iterations         | 3000       |
|    time_elapsed       | 180        |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -41.7      |
|    explained_variance | 0          |
|    learning_rate      | 0.0007     |
| 

KeyboardInterrupt: 

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

### Model 5: SAC

In [46]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [47]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

day: 2892, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 3849067.63
total_reward: 2849067.63
total_cost: 46084.97
total_trades: 47159
Sharpe: 0.733
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 19        |
|    time_elapsed    | 600       |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | 1.03e+03  |
|    critic_loss     | 120       |
|    ent_coef        | 0.174     |
|    ent_coef_loss   | -82.6     |
|    learning_rate   | 0.0001    |
|    n_updates       | 11471     |
|    reward          | 3.2562213 |
----------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 20       |
|    time_elapsed    | 1148     |
|    total_timesteps | 23144    |
| train/             |          |
|    actor_loss      | 564      |
|    critic_loss     | 66       |
|    ent

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [48]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [49]:
insample_risk_indicator.vix.describe()

count    2893.000000
mean       18.824245
std         8.489311
min         9.140000
25%        13.330000
50%        16.139999
75%        21.309999
max        82.690002
Name: vix, dtype: float64

In [50]:
insample_risk_indicator.vix.quantile(0.996)

57.40400183105453

In [51]:
insample_risk_indicator.turbulence.describe()

count    2893.000000
mean       34.574231
std        43.787166
min         0.000000
25%        14.966265
50%        24.123878
75%        39.161926
max       652.502415
Name: turbulence, dtype: float64

In [52]:
insample_risk_indicator.turbulence.quantile(0.996)

276.4520288093791

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [53]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [54]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,89.904594,110737200.0,2.0,3.014603,92.699319,80.179953,62.807144,107.491815,29.730532,83.933767,77.717542,28.620001,53.068508
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,242.083481,6575800.0,2.0,3.665611,234.264608,201.821546,61.279648,271.571268,46.806139,216.584930,217.666556,28.620001,53.068508
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.086372,3301000.0,2.0,-0.389163,110.423942,87.759330,48.504813,-66.328730,3.142448,97.244635,90.695524,28.620001,53.068508
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.068508
0,2020-07-01,CAT,129.380005,129.399994,125.879997,120.651634,2807800.0,2.0,1.272629,130.624290,113.353123,52.865416,35.597277,14.457404,119.412839,113.646677,28.620001,53.068508


In [55]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [56]:
df_account_value.shape

(336, 2)

In [57]:
df_account_value.tail()

,date,account_value
331,2021-10-22,1.429173e+06
332,2021-10-25,1.431737e+06
333,2021-10-26,1.434151e+06
334,2021-10-27,1.430452e+06
335,2021-10-28,1.440208e+06


In [58]:
df_actions.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,76,44,91,11,92,96,97,0,90,7,...,49,80,0,0,90,94,0,93,0,0
2020-07-02,76,44,91,11,92,96,97,0,90,7,...,49,80,0,0,90,94,0,93,0,0
2020-07-06,76,44,91,11,92,96,97,0,90,7,...,49,80,0,0,90,94,0,93,0,0
2020-07-07,76,44,91,11,92,96,97,0,90,7,...,49,80,0,0,90,94,0,93,0,0
2020-07-08,0,0,91,0,92,96,97,0,90,0,...,0,58,0,0,90,94,0,93,0,0


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [59]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.314676
Cumulative returns     0.440208
Annual volatility      0.139391
Sharpe ratio           2.039234
Calmar ratio           3.561024
Stability              0.935059
Max drawdown          -0.088367
Omega ratio            1.414487
Sortino ratio          3.076930
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.109000
Daily value at risk   -0.016434
dtype: float64


In [60]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (336, 8)
Annual return          0.269722
Cumulative returns     0.374922
Annual volatility      0.139083
Sharpe ratio           1.792302
Calmar ratio           3.020136
Stability              0.919220
Max drawdown          -0.089308
Omega ratio            1.347571
Sortino ratio          2.655481
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.052781
Daily value at risk   -0.016534
dtype: float64


In [61]:
df_account_value.loc[0,'date']

'2020-07-01'

In [62]:
df_account_value.loc[len(df_account_value)-1,'date']

'2021-10-28'

<a id='6.2'></a>
## 7.2 BackTestPlot

In [63]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (336, 8)


Start date,2020-07-01
End date,2021-10-28
Total months,16
,Backtest
Annual return,31.5%
Cumulative returns,44.0%
Annual volatility,13.9%
Sharpe ratio,2.04
Calmar ratio,3.56
Stability,0.94
Max drawdown,-8.8%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'